In [3]:
# Importing drive for google colab
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [4]:
# Import os to change directory
import os
os.chdir("/content/mydrive")
!pip install requests-cache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 KB 6.5 MB/s eta 0:00:00


In [5]:
# Import all required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import json
import tqdm
import numpy as np
import requests_cache
import re

# Initialize session using request_cache
session = requests_cache.CachedSession('demo_cache_bookings')

'''
This function scrapes the Hotel details.
Accepts 2 arguments:
link: (str) Link to the hotel to be scrapped
revs: (str) Reviews available for the hotel

Returns a list of reviews, amenities and ratings for different categories
'''
def scrape_hotel(link,revs):
  page_name = (link.split('/hotel/us/')[1]).split('.html')[0] # Get the page name from the link 
  review_list = 0
  review_list = get_reviews(page_name,revs) # Call the funtion to get reviews 
  amenities_list_names = []
  ratings = {}
  time.sleep(0.1)
  response_c = session.get(url = 'https://www.booking.com'+link) # Hit the url for the hotel using link
  soup_details = BeautifulSoup(response_c.text, 'html.parser')
  details_list = soup_details.findAll('div')
  span_list = soup_details.findAll('span')

  # Scrape the list of amentites from the HTML response recieved
  for detail in details_list:
    if (str(detail).startswith('<div data-testid="property-most-popular-facilities-wrapper"')):
      soup_amenities = BeautifulSoup(str(detail), 'html.parser')
      amenities_list = soup_amenities.findAll('span')
      for amenity in amenities_list:
        if str(amenity).startswith('<span class='):
          if ((str(amenity).split('>')[1]).split('</')[0]).startswith('<') :
            pass
          else :
            amenities_list_names.append((str(amenity).split('>')[1]).split('</')[0])
      break
  # Scrape the best rating for the hotel
  scripts = soup_details.findAll('script')
  for script in scripts:
    if str(script).startswith('<script nonce=') and 'type="application/ld+json"' in str(script):
      li = str(script).split('\n')
      for i in li:
        if i.strip().startswith('"bestRating"'):
          ratings['Best Rating'] = float((i.strip().split(':')[1]).split(',')[0])

  # Scrape the list of rating available for each Hotel (Staff, Facilities, Comfort ...)
  rats = ['Staff', 'Facilities', 'Cleanliness', 'Comfort', 'Value for money', 'Location', 'Free WiFi']
  p=0
  for num in range(0, len(span_list)):
    if str(span_list[num]).startswith('<span class="c-score-bar__title">'):
      ratings[rats[p]] = float((str(span_list[num+1]).split('</')[0]).split('>')[1])
      p=p+1
  
  # Return list of reviews, amenities and ratings for different categories
  return ([review_list,amenities_list_names,ratings])

In [6]:
# Function to get the payload for reviews url
def set_payload(offset, rows, pagename, aid='355028', sid='f87a5cba90a64eea25b1f229ea4fd9e1'):
    payload = {
    'aid': aid,
    'sid': sid,
    'srpvid': 'a4a298caac950022',
    'cc1': 'us',
    'pagename': pagename,
    'r_lang': '',
    'review_topic_category_id': '',
    'type': 'total',
    'score': '',
    'sort': '',
    'time_of_year':'', 
    'dist': '1',
    'offset':str(offset),
    'rows': str(rows),
    'rurl': '',
    'text': '',
    'translate':'', 
    '_': '1679348660326'
    }
    return payload

# Function to get header for reviews url
def set_header(cookie):
    headers ={
    'authority': 'www.booking.com',
    'method': 'GET',
    'path': '/reviewlist.en-gb.html',
    'scheme': 'https',
    'accept': '*/*',
    'accept-encodingo': 'gzip, deflate, br',
    'accept-language': 'en-GB,en;q=0.7',
    'cookie': cookie,
    'referer': 'https://www.booking.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'x-booking-aid': '355028',
    'x-booking-client-info': 'THHSOFRURURYNYHIYTLRQJRbWdWOGVO|1,THHSOFRURURYNYHIYTLRQJRbWdWOGVO|3,THHSOFRURURYNYHIYTLRQJRbWdWOGVO|5,aaTBNZZJRLESPIDNJDPVBC|1,aaTBNZZJRLESPIDNJDPVBC|2,adUAAVfDfWZJEHSCGVbSHT|2',
    'x-booking-csrf': '8QsZZAAAAAA=fh8oX3vXdXs-JURUAcIAth4G1ENmQTTB9Q2VI_H5vvAQuF0ZzQ_S1HwN2bIYvuXLdJiHoSqDFEoiX2OkySY_-5IuTkZnkP7VmdCseFXjpKzgubBULW6AnTyNh2HFVlrGMhYQcpIsrek0WB596M9i-U4EtJrGXV5LnBJroFMoc__Bt19r1U08DlIsNruDBOqwoS6TDw-QFN2bH_Dw',
    'x-booking-info': '1680920,1683520,1687140,1689170,1692140,1692950,1696640,1696800,1696860,1699500,1700010,1700580,THHSOFRURURYNYHIYTLRQJRbWdWOGVO|1,THHSOFRURURYNYHIYTLRQJRbWdWOGVO|3,THHSOFRURURYNYHIYTLRQJRbWdWOGVO|5,aaTBNZZJRLESPIDNJDPVBC|1,eWHMcCcCcCFKJBKWUbPNadSFbTdNDNSNC|1,1698510|2,1700010|2,1696800|3,1680920|1,1700010|4,aaTBNZZJRLESPIDNJDPVBC|2,adUAAVfDfWZJEHSCGVbSHT|2,HINZJLeUXSaZbOTMXC|1,HINZJLeUXSaZbdKNKNKPJdBJOTXNORe|1,HINZJLeUXSaZbOTMXC|2,HINZJLeUXSaZbOTMXC|6,HINZJLeUXSaZbdKNKNKPJdBJOTXNORe|2,HINZJLeUXSaZbdKNKNKPJdBJOTXNORe|8,HINZJLeUXSaZbCcBUEXO|5,HINZJLeUXSaZbCcBUEXO|7,HINZJLeUXSaZbOTMXC|3,YdXfMTXEUDdeOYSCaIfWcACVVLZPecOEO|1,YdXfMTXEUDdeOYSCaIfWcACVVLZPecOEO|6,YdXfMTXEUDdeOYSCaIfWcACVVLZPecOEO|3,INFddKNKNKPBDJJHMVGPLTLReASdVLT|2,aXBNTfZHYHQDVCXdUFDeTQQVDaVYEO|1,aXBNTfZHYHQDVCXdUFDeTQQVDaVYEO|3',
    'x-booking-language-code': 'en-gb',
    'x-booking-pageview-id': '8ac998d8834b026a',
    'x-booking-session-id': 'f87a5cba90a64eea25b1f229ea4fd9e1',
    'x-booking-sitetype-id': '1',
    'x-partner-channel-id': '3',
    'x-requested-with': 'XMLHttpRequest'
    }
    return headers

# Function to get headers for hotels searchresults url  
def set_header_a(cookie):
    headers ={
    'authority': 'www.booking.com',
    'method': 'GET',
    'path': '/searchresults.en-us.html',
    'scheme': 'https',
    'accept': 'text/html',
    'accept-encodingo': 'gzip, deflate, br',
    'accept-language': 'en-GB,en;q=0.7',
    'cache-control': 'max-age=0',
    'cookie': cookie,
    'upgrade-insecure-requests': '1',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
    }
    return headers

# Function to get payload for hotels searchresults url
def set_payload_a(city, dest, aid, sid, offset):
    payload = {
    'aid': aid,
    'sid': sid,
    'ss': city,
    'ssne': city,
    'ssne_untouched': city,
    'highlighted_hotels': '374407',
    'lang': 'en-us',
    'sb': '1',
    'src_elem': 'sb',
    'src': 'searchresults',
    'dest_id': dest,
    'dest_type': 'city',
    'checkin': '2023-04-28',
    'checkout': '2023-04-30',
    'group_adults': '2',
    'no_rooms': '1',
    'group_children': '0',
    'sb_travel_purpose': 'leisure',
    'offset': str(offset)
    }
    return payload

In [7]:
'''
This function scrapes the Hotel details.
Accepts 2 arguments:
page_name: (str) hotel for which reviews need to be scrapped
revs: (str) Reviews available for the hotel

Returns a list of reviews
'''
def get_reviews(page_name,revs):
  url='https://www.booking.com/reviewlist.en-gb.html' # Set the reviews URL
  dict_list=[]
  #set the number of pages in this loop
  revs = revs.split(' ')[0].strip()
  revs = re.sub(',', '', revs)
  number_of_pages = int(int(revs)/10)
  for offset in tqdm.tqdm(range(number_of_pages)):
    #Set offset and number of rows here
    payload = set_payload(offset=offset*10, rows=10, pagename=page_name)
    #Set cookie here
    headers = set_header(cookie=
                     'px_init=0; bkng_sso_session=e30; cors_js=1; OptanonConsent=consentCausedByGPCSignalStored=1676350216807&implicitConsentCountry=nonGDPR&implicitConsentDate=1676350578327; bkng_sso_ses=eyJib29raW5nX2dsb2JhbCI6W3siYSI6MSwiaCI6IlpaWXQzN1U2cWxmSG5zWlN6bkJmU3lacnZ0bndBN3IvZjJVd2lreGJVeEUifV19; bkng_sso_auth=CAEQARpWiuKYi1DOHJSuO4hRLUcoob8rBLZ39K9khJGFmTPO4onFDiuuzjOjOmcFBJGuerC/vcR6HYSE9ycFThhsbi9pV/XvIfCvie32L75iYUAamGzwavCmlNAiEMNjyhVSP6kdS4TMEjCNd6s=; BJS=-; _pxhd=kHWt2-Q0inN0bQMXUXG%2FeyUUriMu0j5ZkFfAi%2FqxlWm0D6ATdwCL95PtIIQHmkCBuvs8qXVDDFky6q8Trh8vMw%3D%3D%3Ao8ZNquaJYFJWHfDCrm9zH8MGa9Jell5wENexYptlUz%2FiZPvDD8MHYk%2FX--Z22Ri0A2mN5QrwCOOUaJqmvk1xhNLdJsUIeHfbKemFRhLhKRw%3D; lastSeen=0; bkng=11UmFuZG9tSVYkc2RlIyh9YXSgTtYpR%2F1WOjMvuuinviEMkxKMKaR8RszOeejweDlwtP1u52V2KKHcuZWtAZXTCazM97dd71rXbb02cdIlBFgcQWZEsiJuYMTFSTHwgEN%2F4WK8rNDgBhcNIUGP6ch8GSLQSQolvOnV8yGkdtp5trbYww1Oad21qOwR5g%2BoBEpQZHaR3ehEvUWa75BXgEUL%2Fg%3D%3D',
                    )
    
    time.sleep(0.1)
    r=session.get(url, headers=headers,params=payload) # Hit the reviews URL
    soup=BeautifulSoup(r.text)
    review_list = soup.find_all('li',{'class':'review_list_new_item_block'})
    
    #Looping through the all the reviews
    for i in review_list: 
        try: # Scrape name
            Reviewer_Name=i.find_all('span',{'class':'bui-avatar-block__title'})[0].string
        except:
            Reviewer_Name=None

        try: # Scrape country
            Reviewer_Country=i.find_all('span',{'class':'bui-avatar-block__subtitle'})[0].text.strip(' \n')
        except:
            Reviewer_Country=None

        try: # Scrape Room type
            Room_Type=i.find_all('div',{'class':'bui-list__body'})[0].string.strip(' \n')
        except:
            Room_Type=None

        try: # Scrape Month
            Review_Month=i.find_all('span',{'class':'c-review-block__date'})[0].text.strip(' \n')
        except:
            Review_Month=None

        try: # Scrape stay period
            Stay_Period=i.find_all('div',{'class':'bui-list__body'})[1].text.split('·')[0].strip(' \n')
        except:
            Stay_Period= None

        try: # Scrape review group like 'Family' etc.
            Reviewer_Group_Type=i.find_all('div',{'class':'bui-list__body'})[-1].string.strip(' \n')
        except:
            Reviewer_Group_Type=None

        try: # Scrape start date
            Stay_Start_Date=i.find_all('span',{'class':'c-review-block__date'})[-1].string.strip(' \n').split(':')[-1].strip(' ')
        except:
            Stay_Start_Date=None

        try: # Scrape review heading
            Main_Review=i.find_all('h3',{'class':'c-review-block__title c-review__title--ltr'})[0].string.strip(' \n')
        except:
            Main_Review=None

        try: # Scrape rating for review
            Review_Rating=i.find_all('div',{'class':'bui-review-score__badge'})[0].string.strip(' \n')
        except:
            Review_Rating=None

        try: # Scrape review like
            Review_Likes=i.find_all('span',{'class':'c-review__body'})[0].string
        except:
            Review_Likes=None

        try: # Scrape review dislike
            Review_Dislikes=i.find_all('span',{'class':'c-review__body'})[1].string
        except:
            Review_Dislikes=None
        
        # Add data into a dictionary and append to a list of reviews
        dict_list.append({
            'Reviewer_Name':Reviewer_Name, 
            'Reviewer_Country':Reviewer_Country,
            'Room_Type':Room_Type,
            'Review_Month':Review_Month,
            'Stay_Period':Stay_Period,
            'Stay_Start_Date':Stay_Start_Date,
            'Reviewer_Group_Type':Reviewer_Group_Type,
            'Main_Review':Main_Review,
            'Review_Rating':Review_Rating,
            'Review_Likes':Review_Likes,
            'Review_Dislikes':Review_Dislikes,
        })
  return dict_list # Return list of reviews

In [8]:
'''
This function gets the number of properties loaded for a city
Accepts 1 arguments:
city_: (str) Name of the city you want the list of hotels for 

Returns number of properties loaded
'''
def get_pages(city_):
  url = 'https://www.booking.com/searchresults.en-us.html' # Set the URL
  payload = set_payload_a(city=city_[0], dest=city_[1], aid=city_[2], sid=city_[3], offset=25)
  #Set your cookie here
  headers = set_header(cookie=
                     'px_init=0; bkng_sso_session=e30; cors_js=1; OptanonConsent=consentCausedByGPCSignalStored=1676350216807&implicitConsentCountry=nonGDPR&implicitConsentDate=1676350578327; bkng_sso_ses=eyJib29raW5nX2dsb2JhbCI6W3siYSI6MSwiaCI6IlpaWXQzN1U2cWxmSG5zWlN6bkJmU3lacnZ0bndBN3IvZjJVd2lreGJVeEUifV19; bkng_sso_auth=CAEQARpWiuKYi1DOHJSuO4hRLUcoob8rBLZ39K9khJGFmTPO4onFDiuuzjOjOmcFBJGuerC/vcR6HYSE9ycFThhsbi9pV/XvIfCvie32L75iYUAamGzwavCmlNAiEMNjyhVSP6kdS4TMEjCNd6s=; BJS=-; _pxhd=kHWt2-Q0inN0bQMXUXG%2FeyUUriMu0j5ZkFfAi%2FqxlWm0D6ATdwCL95PtIIQHmkCBuvs8qXVDDFky6q8Trh8vMw%3D%3D%3Ao8ZNquaJYFJWHfDCrm9zH8MGa9Jell5wENexYptlUz%2FiZPvDD8MHYk%2FX--Z22Ri0A2mN5QrwCOOUaJqmvk1xhNLdJsUIeHfbKemFRhLhKRw%3D; lastSeen=0; bkng=11UmFuZG9tSVYkc2RlIyh9YXSgTtYpR%2F1WOjMvuuinviEMkxKMKaR8RszOeejweDlwtP1u52V2KKHcuZWtAZXTCazM97dd71rXbb02cdIlBFgcQWZEsiJuYMTFSTHwgEN%2F4WK8rNDgBhcNIUGP6ch8GSLQSQolvOnV8yGkdtp5trbYww1Oad21qOwR5g%2BoBEpQZHaR3ehEvUWa75BXgEUL%2Fg%3D%3D',
                    )
  time.sleep(0.1)
  response=session.get(url, headers=headers,params=payload) # Hit the url and get the response
  soup = BeautifulSoup(response.text, 'html.parser')

  # Get the number of properties loaded for the givn city
  pages = soup.find_all('div',{'data-capla-component':'b-search-web-searchresults/HeaderDesktop'})
  pages = pages[0].find_all('h1')[0].string
  pages = int((pages.split(':')[1].strip()).split('properties')[0].strip())
  
  return pages # Return the number of properties loaded

In [15]:
'''
This function scrapes the Hotel details.
Accepts 1 arguments:
city_: (str) Name of the city you want the list of hotels for 

Returns the list of hotels for the given city
'''
def scrape_hotels_list(city_):
  #properties_num = get_pages(city_)
  hotel_list = []
  # Run this loop using the number of properties (0, properties_num, 25) and 25 as offset
  for i in range (0,201,25): # This was modified for each run to load data in chunks
    url = 'https://www.booking.com/searchresults.en-us.html' # Set the URL
    payload = set_payload_a(city=city_[0], dest=city_[1], aid=city_[2], sid=city_[3], offset=i)
    #Set your cookie here
    headers = set_header(cookie=
                     'px_init=0; bkng_sso_session=e30; cors_js=1; OptanonConsent=consentCausedByGPCSignalStored=1676350216807&implicitConsentCountry=nonGDPR&implicitConsentDate=1676350578327; bkng_sso_ses=eyJib29raW5nX2dsb2JhbCI6W3siYSI6MSwiaCI6IlpaWXQzN1U2cWxmSG5zWlN6bkJmU3lacnZ0bndBN3IvZjJVd2lreGJVeEUifV19; bkng_sso_auth=CAEQARpWiuKYi1DOHJSuO4hRLUcoob8rBLZ39K9khJGFmTPO4onFDiuuzjOjOmcFBJGuerC/vcR6HYSE9ycFThhsbi9pV/XvIfCvie32L75iYUAamGzwavCmlNAiEMNjyhVSP6kdS4TMEjCNd6s=; BJS=-; _pxhd=kHWt2-Q0inN0bQMXUXG%2FeyUUriMu0j5ZkFfAi%2FqxlWm0D6ATdwCL95PtIIQHmkCBuvs8qXVDDFky6q8Trh8vMw%3D%3D%3Ao8ZNquaJYFJWHfDCrm9zH8MGa9Jell5wENexYptlUz%2FiZPvDD8MHYk%2FX--Z22Ri0A2mN5QrwCOOUaJqmvk1xhNLdJsUIeHfbKemFRhLhKRw%3D; lastSeen=0; bkng=11UmFuZG9tSVYkc2RlIyh9YXSgTtYpR%2F1WOjMvuuinviEMkxKMKaR8RszOeejweDlwtP1u52V2KKHcuZWtAZXTCazM97dd71rXbb02cdIlBFgcQWZEsiJuYMTFSTHwgEN%2F4WK8rNDgBhcNIUGP6ch8GSLQSQolvOnV8yGkdtp5trbYww1Oad21qOwR5g%2BoBEpQZHaR3ehEvUWa75BXgEUL%2Fg%3D%3D',
                    )
    time.sleep(0.1)
    response=session.get(url, headers=headers,params=payload) # Hit the URL and load the response
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.find_all('div',{'data-testid':'property-card'})

    # Loop through each hotel property to scrape the data
    for item in items:
      try: # Scrape hotel name
        name = item.find_all('div',{'data-testid':'title'})[0].string
      except:
        name = None
      try: # Scrape hotel location
        location = item.find_all('span',{'data-testid':'distance'})[0].string
      except:
        location = None
      try: # Scrape hotel price
        price = item.find_all('span',{'data-testid':'price-and-discounted-price'})[0].string
      except:
        price = None
      try: # Scrape room type
        room = item.find_all('div',{'data-testid':'recommended-units'})
        room_type = room[0].find_all('span')[0].string
      except:
        room_type = None
      try: # Scrape link of the hotel
        link = (item.find_all('a',{'data-testid':'title-link'})[0]['href']).split('?')[0]
        link = link.split('https://www.booking.com')[1]
      except:
        link = None
      try: # Scrape review score, review category and total number of reviews for the hotel
        rev_list = item.find_all('a',{'data-testid':'review-score-link'})
        rev_list = rev_list[0].findAll('div')
        review_score = rev_list[1].string
        review_category = rev_list[3]['aria-label']
        revs = rev_list[4].string
      except:
        list_rev = None
        review_score = None
        review_category = None
        revs = None
      flag = 0
      # Check if the hotel is already scrapped, if yes skip it
      if len(hotel_list) != 0:
          for k in hotel_list:
            if k['name'] == name:
              flag = 1
      if flag == 1:
        continue
      if revs != None:
        # If number of reviews is not None, scrape reviews by calling the function
        hotel_details = scrape_hotel(link,revs)
        amenities = hotel_details[1]
        ratings = hotel_details[2]
        review_list = hotel_details[0]
        if name is not None and link is not None:
          # If hotel name and link are not none, add the hotel data into a dictionary and append it to a list
          hotel_list.append({'name':name,
                         'location':location,
                         'price':price,
                         'room_type':room_type,
                         'link':link,
                         'review_score':review_score,
                         'review_category':review_category,
                         'no_of_reviews':revs,
                         'amenities':amenities,
                         'ratings':ratings,
                         'reviews_list':review_list
                         })    
  # Return hotel list
  return hotel_list

In [19]:
# Define cities to be scrapped
cities = [['San Francisco', '20015732', '304142', 'abb6e0ad44a11fc8234d1e153cbab286'], 
          ['Los Angeles, California, United States', '20014181', '304142', 'abb6e0ad44a11fc8234d1e153cbab286'],
          ['San Diego, California, United States', '20015725', '355028', 'abb6e0ad44a11fc8234d1e153cbab286'],
          ['Sacramento, California, United States', '20015688', '355028', 'abb6e0ad44a11fc8234d1e153cbab286'],
          ['Santa Cruz, california, United States', '20015797', '355028', 'abb6e0ad44a11fc8234d1e153cbab286']]

final_hotel_list = {}
hotel_list = scrape_hotels_list(cities[1]) # Scrape city 1 by 1 as the huge amount of daa is crashing the RAM

# Add data into dictionary
final_hotel_list['city_name'] = cities[1][0]
final_hotel_list['hotel_list'] = hotel_list

100%|██████████| 10/10 [00:01<00:00,  8.92it/s]


In [22]:
# Export data from dictionary into a JSON file
# File name set as Scrape_data_<city name><num> num is 1,2,3... for each city
with open('Scrape_data_LA1.json', 'w') as f: # Change JSON name for each city while loading data in chunks
  json.dump(final_hotel_list, f, indent=4)